# Layers

In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
# export
# default_exp layers
from faimed3d.basics import *
from faimed3d.augment import *
from faimed3d.preprocess import *
from faimed3d.models.resnet import resnet50_3d
from faimed3d.data import *
from fastai.vision.learner import _default_meta, _add_norm, model_meta, create_body, has_pool_type, _get_first_layer, _load_pretrained_weights
from fastai.layers import ResBlock

In [ ]:
# export
from fastai.basics import *
from fastai.callback.all import *

In [ ]:
d = pd.read_csv('../data/radiopaedia_cases.csv')
dls = ImageDataLoaders3D.from_df(d, 
                                 item_tfms = ResizeCrop3D(crop_by = (0., 0.1, 0.1), resize_to = (20, 150, 150), perc_crop = True),
                                 bs = 2, 
                                 val_bs = 2)

## Helper functions
Some functions from `fastai.layers` are needed to construct learners (see next notebook). For this some slight modifications had to be made. 
The `in_channel` function had to be modified to also accept 3D models wich have 5D weights and the `num_features_model` function was adapted to pass a size tuple of len 3 instead of 2. The other functions were not changed but copied to avoid conflicts when loaded directly from fastai.   
`cnn_learner_3d` is essentially the same function as fastais `cnn_learner`, just adds a new callback. 

In [ ]:
# export
def create_body(arch, n_in=3, pretrained=True, cut=None):
    "Cut off the body of a typically pretrained `arch` as determined by `cut`"
    model = arch(pretrained=pretrained)
    _update_first_layer(model, n_in, pretrained)
    #cut = ifnone(cut, cnn_config(arch)['cut'])
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if   isinstance(cut, int):      return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or a function")

In [ ]:
# export
def _update_first_layer(model, n_in, pretrained):
    "Change first layer based on number of input channels"
    if n_in == 3: return
    first_layer, parent, name = _get_first_layer(model)
    assert isinstance(first_layer, nn.Conv3d), f'Change of input channels only supported with Conv3d, found {first_layer.__class__.__name__}. Use the fastai implementation'
    assert getattr(first_layer, 'in_channels') == 3, f'Unexpected number of input channels, found {getattr(first_layer, "in_channels")} while expecting 3'
    params = {attr:getattr(first_layer, attr) for attr in 'out_channels kernel_size stride padding dilation groups padding_mode'.split()}
    params['bias'] = getattr(first_layer, 'bias') is not None
    params['in_channels'] = n_in
    new_layer = nn.Conv3d(**params)
    if pretrained:
        _load_pretrained_weights(new_layer, first_layer)
    setattr(parent, name, new_layer)

fastai `create_body` can adapt the number of input channels but only for 2d convolutions. With slight changes in the code of the two fastai functions, it can be adapted to work with 3d convolutions. 

In [ ]:
body_3d = create_body(resnet50_3d, pretrained=True, n_in=2)

In [ ]:
# export
def in_channels(m):
    """
    Return the shape of the first weight layer in `m`.
    same as fastai.vision.learner.in_channels but allows l.weight.ndim of 4 and 5
    """
    for l in flatten_model(m):
        if getattr(l, 'weight', None) is not None and l.weight.ndim in [4,5]:
            return l.weight.shape[1]
    raise Exception('No weight layer')

`in_channels` from fastai only returns a result if `weight.ndim == 4` but in 3D convolutional layers, it will be 5 dimensions, so the functions has to be adapted. 

In [ ]:
test_eq(in_channels(body_3d), 2)

`num_features_model` is unchanged, but needs to be defined here to correctly call the adapted `in_channels` function

In [ ]:
# export
def num_features_model(m):
    """
    Return the number of output features for `m`.
    same as fastai.vision.learner.num_features_model passes model_size a len 3 tuple of sz

    """
    sz,ch_in = 32,in_channels(m)
    while True:
        #Trying for a few sizes in case the model requires a big input size.
        return model_sizes(m, (sz,sz,sz))[-1][1]
        try:
            return model_sizes(m, (sz,sz,sz))[-1][1]
        except Exception as e:
            sz *= 2
            print(sz)
            if sz > 2048: raise e

In [ ]:
# export
def model_sizes(m, size=(8,64,64)):
    "Pass a dummy input through the model `m` to get the various sizes of activations. same as fastai func"
    with hook_outputs(m) as hooks:
        _ = dummy_eval(m, size=size)
        return [o.stored.shape for o in hooks]

In [ ]:
# export
def dummy_eval(m, size=(8,64,64)):
    "Evaluate `m` on a dummy input of a certain `size`. Same as fastai func"
    ch_in = in_channels(m)
    x = one_param(m).new(1, ch_in, *size).requires_grad_(False).uniform_(-1.,1.)
    with torch.no_grad(): return m.eval()(x)

In [ ]:
dummy_eval(body_3d).size()
model_sizes(body_3d)

[torch.Size([1, 128, 9, 21, 21]),
 torch.Size([1, 256, 9, 21, 21]),
 torch.Size([1, 512, 5, 11, 11]),
 torch.Size([1, 1024, 3, 6, 6]),
 torch.Size([1, 2048, 2, 3, 3])]

In [ ]:
test_eq(num_features_model(body_3d), 2048)

`create_cnn_model` is unchanged, but needs to be redefined to correctly call `num_features_model` which then calls the changed `in_channels` function

In [ ]:
# export
class Concat(Module):
    
    def __init__(self, ni, ndim, dim = 1):
        store_attr()
        self.bn = BatchNorm(ni, ndim)
        self.act = nn.ReLU()

    def forward(self, inputs:(list, tuple))->Tensor:
        inputs = torch.cat(inputs, self.dim)
        return self.act(self.bn(inputs))

`fastai` performes adaptive concat pooling as first step in the new header, which is adapted to 3D. 

In [ ]:
# export
class AdaptiveConcatPool3d(Module):
    "Layer that concats `AdaptiveAvgPool3d` and `AdaptiveMaxPool3d`"
    def __init__(self, size=None):
        self.size = size or 1
        self.ap = nn.AdaptiveAvgPool3d(self.size)
        self.mp = nn.AdaptiveMaxPool3d(self.size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [ ]:
# export
def create_head(nf, n_out, lin_ftrs=None, ps=0.5, concat_pool=True, bn_final=False, lin_first=False, y_range=None):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    ps = L(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    pool = AdaptiveConcatPool3d() if concat_pool else nn.AdaptiveAvgPool3d(1)
    layers = [pool, Flatten()]
    if lin_first: layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
        layers += LinBnDrop(ni, no, bn=True, p=p, act=actn, lin_first=lin_first)
    if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    if y_range is not None: layers.append(SigmoidRange(*y_range))
    return nn.Sequential(*layers)

`create_head` is the same as fastai function, but used 3d pooling. 

In [ ]:
# export
def create_cnn_model_3d(arch, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom convnet architecture using `arch`, `n_in` and `n_out`. Identical to fastai func"
    body = create_body(arch, n_in, pretrained, cut)
    if custom_head is None:
        nf = num_features_model(body) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

`create_cnn_model_3d` is similar to `create_cnn_model`.

In [ ]:
model = create_cnn_model_3d(resnet50_3d, 2, 1, pretrained = False)

In [ ]:
model(torch.randn(2, 3, 3, 10, 10)).size()

torch.Size([2, 2])

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_preprocessing.ipynb.
Converted 03_transforms.ipynb.
Converted 04_dataloaders.ipynb.
Converted 05_layers.ipynb.
Converted 06_learner.ipynb.
Converted 06a_models.alexnet.ipynb.
Converted 06b_models.resnet.ipynb.
Converted 06c_model.efficientnet.ipynb.
Converted 06d_models.unet.ipynb.
Converted 06e_models.deeplabv3.ipynb.
Converted 06f_models.losses.ipynb.
Converted 07_callback.ipynb.
Converted index.ipynb.


###### 